# EXPLORATION DES DONNÉES DES POLLUANTS

## Objectifs :

- Extraire la liste des polluants recherchés dans les prélèvements
- Sortir des premières analyses de ces données
- Disponibilité de la donnée
- Couverture du territoire : pour chaque polluant, quelle est la part des UDI avec analyse de chaque polluant
- Périodicité : pour chaque polluant, à quelle fréquence collecte-t-on ?
- Visualisation sur carte de la donnée pour chaque polluant. (obj : avoir une idée de la distribution, des valeurs min max etc)
- Analyse des polluants par seuils de dépassement

<p> => Analyser la donnée pour chaque polluant</p><br> 

---

<br>
<br>

> # Première étape : comprendre la donnée et les tables disponibles

In [2]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données et par connecter la base de données DuckDB

import duckdb

con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [86]:
# Liste des différentes tables présentes dans la base de données

tables = con.execute("SHOW TABLES").fetch_df()
tables

,name
0,edc_communes
1,edc_prelevements
2,edc_resultats


In [87]:
# Comprendre la data dans chaque table

# Table edc_communes
con.execute("DESCRIBE edc_communes").fetchdf()

,column_name,column_type,null,key,default,extra
0,inseecommune,VARCHAR,YES,None,None,None
1,nomcommune,VARCHAR,YES,None,None,None
2,quartier,VARCHAR,YES,None,None,None
3,cdreseau,VARCHAR,YES,None,None,None
4,nomreseau,VARCHAR,YES,None,None,None
5,debutalim,DATE,YES,None,None,None
6,de_partition,INTEGER,YES,None,None,None
7,de_ingestion_date,DATE,YES,None,None,None


In [88]:
# Table edc_prelevements

con.execute("DESCRIBE edc_prelevements").fetchdf()

,column_name,column_type,null,key,default,extra
0,cddept,VARCHAR,YES,None,None,None
1,cdreseau,VARCHAR,YES,None,None,None
2,inseecommuneprinc,VARCHAR,YES,None,None,None
3,nomcommuneprinc,VARCHAR,YES,None,None,None
4,cdreseauamont,VARCHAR,YES,None,None,None
5,nomreseauamont,VARCHAR,YES,None,None,None
6,pourcentdebit,VARCHAR,YES,None,None,None
7,referenceprel,VARCHAR,YES,None,None,None
8,dateprel,DATE,YES,None,None,None
9,heureprel,VARCHAR,YES,None,None,None


In [89]:
# Table edc_resultats

con.execute("DESCRIBE edc_resultats").fetchdf()

,column_name,column_type,null,key,default,extra
0,cddept,VARCHAR,YES,None,None,None
1,referenceprel,VARCHAR,YES,None,None,None
2,cdparametresiseeaux,VARCHAR,YES,None,None,None
3,cdparametre,BIGINT,YES,None,None,None
4,libmajparametre,VARCHAR,YES,None,None,None
5,libminparametre,VARCHAR,YES,None,None,None
6,libwebparametre,VARCHAR,YES,None,None,None
7,qualitparam,VARCHAR,YES,None,None,None
8,insituana,VARCHAR,YES,None,None,None
9,rqana,VARCHAR,YES,None,None,None


In [90]:
# Aperçu de la table "edc_communes"

con.execute("SELECT * FROM edc_communes LIMIT 5").fetch_df()

,inseecommune,nomcommune,quartier,cdreseau,nomreseau,debutalim,de_partition,de_ingestion_date
0,01001,ABERGEMENT-CLEMENCIAT (L'),-,001000556,BDS ST DIDIER/CHALARONNE,2010-09-07,2024,2025-02-07
1,01002,ABERGEMENT-DE-VAREY (L'),-,001000369,L'ABERGEMENT-DE-VAREY,2010-09-07,2024,2025-02-07
2,01004,AMBERIEU-EN-BUGEY,Vareilles,001000248,AMBERIEU VAREILLES,2010-09-07,2024,2025-02-07
3,01004,AMBERIEU-EN-BUGEY,St Germain_Brédevent,001000249,AMBERIEU SAINT GERMAIN DOUVRES,2010-09-07,2024,2025-02-07
4,01004,AMBERIEU-EN-BUGEY,ville,001000251,AMBERIEU VILLE,2010-09-07,2024,2025-02-07


In [91]:
# Aperçu de la table "edc_prelevements"

con.execute("SELECT * FROM edc_prelevements LIMIT 5").fetch_df()

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000003,01007,AMBRONAY,None,None,None,00100139034,2024-01-23,11h27,Eau d'alimentation conforme aux limites de qua...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,N,C,2024,2025-02-07
1,001,001000003,01007,AMBRONAY,None,None,None,00100139393,2024-02-13,12h49,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07
2,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100139969,2024-03-13,10h46,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07
3,001,001000003,01007,AMBRONAY,None,None,None,00100140209,2024-04-05,07h27,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07
4,001,001000003,01007,AMBRONAY,None,None,None,00100140543,2024-05-13,12h43,Eau d'alimentation conforme aux exigences de q...,SYND. EAUX REGION D'AMBERIEU-EN-B,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,SERA - SYNDICAT DES EAUX DE LA REGION D'AMBERI...,C,C,C,C,2024,2025-02-07


In [3]:
# Aperçu de la table "edc_resultats"

con.execute("SELECT * FROM edc_resultats LIMIT 5").fetch_df()

,cddept,referenceprel,cdparametresiseeaux,cdparametre,libmajparametre,libminparametre,libwebparametre,qualitparam,insituana,rqana,cdunitereferencesiseeaux,cdunitereference,limitequal,refqual,valtraduite,casparam,referenceanl,de_partition,de_ingestion_date
0,001,00100119085,12DCLE,1161,"DICHLOROÉTHANE-1,2","Dichloroéthane-1,2",None,N,L,"<0,50",µg/L,133,<=3 µg/L,None,0.000,107-06-2,00100125759,2020,2025-02-12
1,001,00100119085,A2H,1832,ATRAZINE-2-HYDROXY,Atrazine-2-hydroxy,None,N,L,"<0,020",µg/L,133,"<=0,1 µg/L",None,0.000,2163-68-0,00100125759,2020,2025-02-12
2,001,00100119085,ACRYL,1457,ACRYLAMIDE,Acrylamide,None,N,L,"<0,10",µg/L,133,<=0.1 µg/L,None,0.000,79-06-1,00100125759,2020,2025-02-12
3,001,00100119085,ACTIK40,1036,ACTIVITÉ BÊTA ATTRIBUABLE AU K40,Activité bêta attribuable au K40,None,N,L,"0,034",Bq/L,9,None,None,0.034,None,00100125759,2020,2025-02-12
4,001,00100119085,ACTITR,2098,ACTIVITÉ TRITIUM (3H),Activité Tritium (3H),None,N,L,<8,Bq/L,9,None,<=100 Bq/L,0.000,None,00100125759,2020,2025-02-12


> # Deuxième étape : réaliser des analyses simples pour comprendre les datasets

### 1 - Analyse de la table edc_communes

In [4]:
# Chargeons la table edc_communes dans un pandas dataframe, et calculons le nombre de communes

communes = con.table("edc_communes").to_df()
nombre_de_communes = communes.nunique()["inseecommune"]
print(f"nombre_de_communes = {nombre_de_communes}")

# Chargeons la table edc_communes dans un pandas dataframe, et calculons le nombre de réseaux uniques

reseau = con.table("edc_communes").to_df()
nb_reseaux_uniques = reseau.nunique()["cdreseau"]
print(f"nombre_de_reseaux = {nb_reseaux_uniques}")

nombre_de_communes = 34914
nombre_de_reseaux = 23767


In [5]:
# Combien de communes déservies par chaque réseau ?

query = """
SELECT cdreseau, nomreseau, COUNT(DISTINCT inseecommune) AS nb_communes
FROM edc_communes
GROUP BY cdreseau, nomreseau
ORDER BY nb_communes DESC;
"""

communes_par_reseau = con.execute(query).fetch_df()
communes_par_reseau

,cdreseau,nomreseau,nb_communes
0,032000434,BAROUSSE COMMINGES SAVE,80
1,059000481,EBBLINGHEM,72
2,031000096,MONTAGNE NOIRE,61
3,031000109,ST NERE TROUBAT CLARAC VILLENEUVE,61
4,057001421,SEBVF 2,57
...,...,...,...
24775,067001346,SDEA-SECTEUR NEUENMATTEN-GRENDELBRUCH,1
24776,068006327,DURMENACH,1
24777,068000935,WERENTZHOUSE,1
24778,073000876,RESEAU DES TEPPES,1


In [6]:
# Combien de réseaux desservent > 10 communes ?

where_clause = "HAVING nb_communes > 10"

query = f"""
SELECT cdreseau, nomreseau, COUNT(DISTINCT inseecommune) AS nb_communes
FROM edc_communes
GROUP BY cdreseau, nomreseau
{where_clause}
ORDER BY nb_communes DESC;
"""

reseau_desservant_plus_de_10_communes = con.execute(query).fetch_df()
reseau_desservant_plus_de_10_communes

# % de réseaux desservant > 10 communes sur le total des réseaux

nb_reseaux_desservant_plus_de_10_communes = reseau_desservant_plus_de_10_communes.shape[
    0
]

pourcentage_reseaux_desservant_plus_de_10_communes = (
    nb_reseaux_desservant_plus_de_10_communes / nb_reseaux_uniques * 100
)
print(
    f"pourcentage reseaux desservant plus de 10 communes = {pourcentage_reseaux_desservant_plus_de_10_communes:.2f}%"
)

# Combien de réseaux desservent uniquement une seule commune ?

where_clause = "HAVING nb_communes = 1"

query = f"""
SELECT cdreseau, nomreseau, COUNT(DISTINCT inseecommune) AS nb_communes
FROM edc_communes
GROUP BY cdreseau, nomreseau
{where_clause}
ORDER BY nb_communes DESC;
"""

reseau_desservant_une_seule_commune = con.execute(query).fetch_df()
reseau_desservant_une_seule_commune

# % de réseaux desservant une seule commune sur le total des réseaux

nb_reseaux_desservant_une_seule_commune = reseau_desservant_une_seule_commune.shape[0]
pourcentage_reseaux_desservant_une_seule_commune = (
    nb_reseaux_desservant_une_seule_commune / nb_reseaux_uniques * 100
)
print(
    f"pourcentage reseaux desservant seulement 1 commune = {pourcentage_reseaux_desservant_une_seule_commune:.2f}%"
)


pourcentage reseaux desservant plus de 10 communes = 2.68%
pourcentage reseaux desservant seulement 1 commune = 79.86%


La grande majorité des réseaux déservent une commune unique (80%). Seulement 3% des réseaux desservent plus de 10 communes.


In [7]:
# Y a-t-il des communes desservies par plusieurs réseaux ?

query = """
SELECT inseecommune, COUNT(DISTINCT cdreseau) AS nb_reseaux
FROM edc_communes
GROUP BY inseecommune
HAVING nb_reseaux > 1
ORDER BY nb_reseaux DESC;
"""

communes_desservies_par_plusieurs_reseaux = con.execute(query).fetch_df()
communes_desservies_par_plusieurs_reseaux

,inseecommune,nb_reseaux
0,48027,20
1,73257,19
2,97353,18
3,48009,18
4,22046,18
...,...,...
7016,41149,2
7017,45089,2
7018,46033,2
7019,46117,2


La relation réseau <> commune est une relation many to many. Un réseau peut desservir plusieurs communes. Une commune peut être desservie par plusieurs réseaux. La majorité des communes ont un unique réseau qui les dessert.

- Comment faire pour la cartographie ? Si une commune a plusieurs réseaux et que la qualité de l'eau de ces derniers est hétérogène ?

-> Si un réseau mauvais alors toute la commune marquée à risque ?  
-> Faire la cartographie de manière plus précise que par communes ?

### 2 - Analyse de la table edc_prelevements

In [8]:
# Informations de base sur le tableau edc_prelevements

query = """
SELECT COUNT(*) AS nb_prelevements,
         COUNT(DISTINCT referenceprel) AS nb_prelevements_uniques,
         COUNT(DISTINCT cdreseau) AS nb_reseaux,
         COUNT(DISTINCT cdreseauamont) AS nb_reseaux_amont
FROM edc_prelevements;
"""

informations_prelevements = con.execute(query).fetch_df()
informations_prelevements

,nb_prelevements,nb_prelevements_uniques,nb_reseaux,nb_reseaux_amont
0,2083345,1436114,23767,22897


In [9]:
# Analyse des données des réseaux uniques

query = """
SELECT DISTINCT cdreseau, COUNT(DISTINCT referenceprel) AS nb_prelevements
FROM edc_prelevements
GROUP BY cdreseau
ORDER BY nb_prelevements DESC;
"""

prelevements = con.execute(query).fetch_df()
prelevements


,cdreseau,nb_prelevements
0,069000069,7413
1,069000229,7091
2,092000054,6845
3,069003595,6808
4,069000270,6718
...,...,...
23762,973000588,1
23763,088006607,1
23764,005005327,1
23765,040004097,1


In [ ]:
# Combien de prélevements en moyenne par réseau depuis 2020 ?

query = """
SELECT AVG(nb_prelevements) AS moyenne_prelevements_par_reseau
FROM (
    SELECT cdreseau, COUNT(DISTINCT referenceprel) AS nb_prelevements
    FROM edc_prelevements
    GROUP BY cdreseau
);
"""

moyenne_prelevements_par_reseau = con.execute(query).fetch_df()
print(
    f"Moyenne de prélèvements par réseau: {moyenne_prelevements_par_reseau.iloc[0, 0]:.2f}"
)

# Quelle est la médiane du nombre de prélevements par réseau depuis 2020 ?

query = """
SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY nb_prelevements) AS mediane_prelevements_par_reseau
FROM (
    SELECT cdreseau, COUNT(DISTINCT referenceprel) AS nb_prelevements
    FROM edc_prelevements
    GROUP BY cdreseau
);
"""

mediane_prelevements_par_reseau = con.execute(query).fetch_df()
print(
    f"Médiane de prélèvements par réseau: {mediane_prelevements_par_reseau.iloc[0, 0]:.2f}"
)

Moyenne de prélèvements par réseau: 87.66
Médiane de prélèvements par réseau: 45.00


In [12]:
# Essayons de savoir si les réseaux avec le plus de prélèvements sont ceux de départements fortement peuplés / de grandes métropoles

query = """
SELECT c.inseecommune, c.nomcommune, COUNT(DISTINCT p.referenceprel) AS nb_prelevements
FROM edc_prelevements p
JOIN edc_communes c ON p.inseecommuneprinc = c.inseecommune
GROUP BY c.inseecommune, c.nomcommune
ORDER BY nb_prelevements DESC;
"""

prelevements_par_commune = con.execute(query).fetch_df()
prelevements_par_commune

# Ne garder que les résultats pour les communes avec plus de 1000 prélèvements

prelevements_par_commune = prelevements_par_commune[
    prelevements_par_commune["nb_prelevements"] > 1000
]
prelevements_par_commune

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,inseecommune,nomcommune,nb_prelevements
0,75056,PARIS,8070
1,13055,MARSEILLE,6750
2,95394,MERY-SUR-OISE,3543
3,31555,TOULOUSE,3240
4,06088,NICE,3218
5,94022,CHOISY-LE-ROI,2990
6,44109,NANTES,2894
7,69123,LYON,2749
8,34172,MONTPELLIER,2488
9,33063,BORDEAUX,2306


In [ ]:
# Vérifier si le couple (cdreseau, referenceprel) est unique

query = """
SELECT COUNT(*) AS nb_doublons
FROM (
    SELECT cdreseau, referenceprel, COUNT(*) AS nb_occurrences
    FROM edc_prelevements
    GROUP BY cdreseau, referenceprel
    HAVING nb_occurrences > 1
);
"""

doublons_cdreseau_referenceprel = con.execute(query).fetch_df()
doublons_cdreseau_referenceprel

,nb_doublons
0,1


In [15]:
# Identifier ce doublon

query = """
SELECT p.*
FROM edc_prelevements p
JOIN (
    SELECT cdreseau, referenceprel
    FROM edc_prelevements
    GROUP BY cdreseau, referenceprel
    HAVING COUNT(*) > 1
) s
ON p.cdreseau = s.cdreseau AND p.referenceprel = s.referenceprel;
"""

doublons_cdreseau_referenceprel = con.execute(query).fetch_df()
doublons_cdreseau_referenceprel

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,066,066000125,66021,BOMPAS,None,None,None,06600184310,2021-07-01,14h32,Eau d'alimentation non conforme aux exigences ...,COM URBAIN PERP MEDITERRANEE VEOLIA,VEOLIA EAU CGE PERPIGNAN,COM URBAINE PERPIGNAN MEDITERRANEE,C,C,C,N,2021,2025-02-12
1,066,066000125,66021,BOMPAS,None,None,None,06600184310,2024-07-02,09h15,Eau d'alimentation non conforme aux exigences ...,CATALANE DES EAUX SECTEUR CENTRE,CATALANE DES EAUX - EAU AGGLO,CATALANE DES EAUX - EAU AGGLO,C,C,C,C,2024,2025-02-12


À remonter à l'équipe

In [16]:
# Trouver les referenceprel qui sont dupliquées

query = """
SELECT referenceprel, COUNT(*) AS nb_occurrences
FROM edc_prelevements
GROUP BY referenceprel
HAVING nb_occurrences > 1
ORDER BY nb_occurrences DESC;
"""

doublons_referenceprel = con.execute(query).fetch_df()
doublons_referenceprel

,referenceprel,nb_occurrences
0,01100163162,66
1,01100148387,66
2,01100157022,66
3,01100176795,66
4,01100163166,66
...,...,...
291179,02B00115211,2
291180,02B00115215,2
291181,02B00116491,2
291182,02B00116494,2


In [17]:
# Lister tous les UDI

query = """
SELECT DISTINCT cdreseau AS udi
FROM edc_prelevements
ORDER BY udi
;
"""

prelevements_liste_udi = con.execute(query).fetch_df()
prelevements_liste_udi

,udi
0,001000003
1,001000235
2,001000241
3,001000244
4,001000248
...,...
23762,976003487
23763,976003489
23764,976003554
23765,976003881


In [18]:
# Liste des UDI "root" sur lesquels des prélèvements ont été réalisés au moins une fois

query = """
SELECT DISTINCT cdreseau AS root_udi
FROM edc_prelevements
WHERE cdreseauamont IS NULL
ORDER BY 1;
"""

prelevements_liste_root_udi = con.execute(query).fetch_df()
prelevements_liste_root_udi

,root_udi
0,001000003
1,001000235
2,001000241
3,001000244
4,001000248
...,...
23669,976003487
23670,976003489
23671,976003554
23672,976003881


In [ ]:
# Liste des UDI qui ont fait l'objet de prélèvements extrapolés au moins une fois

query = """
SELECT DISTINCT cdreseau AS extra_udi
FROM edc_prelevements
WHERE cdreseauamont IS NOT NULL
ORDER BY 1;
"""

prelevements_liste_ext_udi = con.execute(query).fetch_df()
prelevements_liste_ext_udi

,extra_udi
0,001000003
1,001000235
2,001000241
3,001000244
4,001000248
...,...
22560,976003464
22561,976003486
22562,976003487
22563,976003489


In [22]:
# Liste des prélèvements sur UDI "root", qui n'ont pas de valeurs sur les champs "cdreseauamont", "nomreseauamont", "pourcentdebit"
# Test effectué avec IS NULL sur tous les champs. Résultat identique avec juste IS NULL sur "cdreseauamont"

query = """
SELECT *
FROM edc_prelevements
WHERE cdreseauamont IS NULL;
"""

prelevements_udi_source = con.execute(query).fetch_df()
prelevements_udi_source

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000003,01007,AMBRONAY,None,None,None,00100119766,2020-02-13,11h40,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
1,001,001000003,01007,AMBRONAY,None,None,None,00100120717,2020-05-14,11h54,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
2,001,001000003,01007,AMBRONAY,None,None,None,00100121072,2020-06-11,12h00,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
3,001,001000003,01007,AMBRONAY,None,None,None,00100121547,2020-07-08,11h09,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
4,001,001000003,01007,AMBRONAY,None,None,None,00100122035,2020-08-13,10h44,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085830,976,976003554,97615,PAMANDZI,None,None,None,97600031877,2024-07-09,10h20,Eau d'alimentation conforme aux exigences de q...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,C,C,C,C,2024,2025-02-12
1085831,976,976003554,97615,PAMANDZI,None,None,None,97600031878,2024-07-09,09h52,Eau d'alimentation conforme aux exigences de q...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,C,C,C,C,2024,2025-02-12
1085832,976,976003554,97615,PAMANDZI,None,None,None,97600031996,2024-07-30,09h52,Eau d'alimentation conforme aux exigences de q...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,C,C,C,C,2024,2025-02-12
1085833,976,976003554,97615,PAMANDZI,None,None,None,97600031997,2024-08-06,09h10,Eau d'alimentation conforme aux exigences de q...,MAYOTTE,SMAE,LES EAUX DE MAYOTTE,C,C,C,C,2024,2025-02-12


In [23]:
# Controle du nombre de referenceprel uniques

prelevements_udi_source["referenceprel"].nunique()

1085834

1 de différence lié au doublon du referenceprel '06600184310'

In [24]:
# Trouver le nombre de prélèvements uniques qui ne sont pas dans la query précédente (liste des prélèvements sur UDI "source")

query = """
SELECT DISTINCT referenceprel
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
    FROM edc_prelevements
    WHERE cdreseauamont IS NULL
);
"""

prelevements_udi_non_source = con.execute(query).fetch_df()
prelevements_udi_non_source

,referenceprel
0,00100119840
1,00100123741
2,00100119564
3,00100122089
4,00100122642
...,...
350275,01300248505
350276,01300242781
350277,01300248803
350278,97600031066


In [ ]:
# Nb de referenceprel uniques UDI "source" + Nb de referenceprel uniques hors UDI "source"
# Sachant que le nb de prélèvements uniques = 1436114

print(f"Nb prel UDI source + nb prel UDI non source = {1085834 + 350280}")

Nb prel UDI source + nb prel UDI non source = 1436114


In [26]:
# Analyse de ces prélèvements sur UDI "non source"
# Comprendre les doublons de referenceprel. Peut-on identifier une ligne unique à garder ?

query = """
SELECT *
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
    FROM edc_prelevements
    WHERE cdreseauamont IS NULL)
ORDER BY referenceprel
LIMIT 30;
"""

test_query = con.execute(query).fetch_df()
test_query

# Ne garder que les lignes avec doublons sur "referenceprel" via pandas

duplicate_refs = test_query["referenceprel"][
    test_query["referenceprel"].duplicated(keep=False)
]
test_query_duplicates = test_query[test_query["referenceprel"].isin(duplicate_refs)]
test_query_duplicates = test_query_duplicates.sort_values(["referenceprel", "dateprel"])

test_query_duplicates

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
4,001,001000618,01283,OYONNAX,001000613,TTP (CLG) HBA OYONNAX,100 %,00100119542,2020-01-13,09h20,Eau d'alimentation conforme aux exigences de q...,HT BUGEY AGGLOMERATION SAUR,SAUR LIMONEST,HAUT BUGEY AGGLOMERATION,C,C,C,C,2020,2025-02-12
5,001,001000614,01283,OYONNAX,001000613,TTP (CLG) HBA OYONNAX,100 %,00100119542,2020-01-13,09h20,Eau d'alimentation conforme aux exigences de q...,HT BUGEY AGGLOMERATION SAUR,SAUR LIMONEST,HAUT BUGEY AGGLOMERATION,C,C,C,C,2020,2025-02-12
6,001,001000511,01283,OYONNAX,001000613,TTP (CLG) HBA OYONNAX,100 %,00100119542,2020-01-13,09h20,Eau d'alimentation conforme aux exigences de q...,HT BUGEY AGGLOMERATION SAUR,SAUR LIMONEST,HAUT BUGEY AGGLOMERATION,C,C,C,C,2020,2025-02-12
22,001,001000677,01385,SAINT-REMY,001000674,TTP (CLG) VEYLE REYSSOUZE H.S.,100 %,00100119558,2020-01-27,09h45,Eau d'alimentation conforme aux exigences de q...,SI VEYLE REYSSOUZE VIEUX-JONC,AQUALTER EXPLOITATION CHARTRES,SIE VEYLE REYSSOUZE VIEUX-JONC,C,C,C,C,2020,2025-02-12
23,001,001001124,01385,SAINT-REMY,001000674,TTP (CLG) VEYLE REYSSOUZE H.S.,100 %,00100119558,2020-01-27,09h45,Eau d'alimentation conforme aux exigences de q...,SI VEYLE REYSSOUZE VIEUX-JONC,AQUALTER EXPLOITATION CHARTRES,SIE VEYLE REYSSOUZE VIEUX-JONC,C,C,C,C,2020,2025-02-12
24,001,001000392,01099,CHAZEY-SUR-AIN,001000306,TTP (CLG) SYND.MIXT.PLAINE DE L'AIN,100 %,00100119559,2020-01-28,08h55,Eau d'alimentation conforme aux exigences de q...,SYND. MIXTE DE LA PLAINE DE L'AIN,SAUR LIMONEST,SYND.MIXTE DE LA PLAINE DE L'AIN,C,C,C,C,2020,2025-02-12
25,001,001000307,01099,CHAZEY-SUR-AIN,001000306,TTP (CLG) SYND.MIXT.PLAINE DE L'AIN,100 %,00100119559,2020-01-28,08h55,Eau d'alimentation conforme aux exigences de q...,SYND. MIXTE DE LA PLAINE DE L'AIN,SAUR LIMONEST,SYND.MIXTE DE LA PLAINE DE L'AIN,C,C,C,C,2020,2025-02-12
26,001,001003906,01099,CHAZEY-SUR-AIN,001000306,TTP (CLG) SYND.MIXT.PLAINE DE L'AIN,100 %,00100119559,2020-01-28,08h55,Eau d'alimentation conforme aux exigences de q...,SYND. MIXTE DE LA PLAINE DE L'AIN,SAUR LIMONEST,SYND.MIXTE DE LA PLAINE DE L'AIN,C,C,C,C,2020,2025-02-12
27,001,001000304,01099,CHAZEY-SUR-AIN,001000306,TTP (CLG) SYND.MIXT.PLAINE DE L'AIN,100 %,00100119559,2020-01-28,08h55,Eau d'alimentation conforme aux exigences de q...,SYND. MIXTE DE LA PLAINE DE L'AIN,SAUR LIMONEST,SYND.MIXTE DE LA PLAINE DE L'AIN,C,C,C,C,2020,2025-02-12
28,001,001000305,01099,CHAZEY-SUR-AIN,001000306,TTP (CLG) SYND.MIXT.PLAINE DE L'AIN,100 %,00100119559,2020-01-28,08h55,Eau d'alimentation conforme aux exigences de q...,SYND. MIXTE DE LA PLAINE DE L'AIN,SAUR LIMONEST,SYND.MIXTE DE LA PLAINE DE L'AIN,C,C,C,C,2020,2025-02-12


In [27]:
# Est-ce que les reseaux en amont qui n'apparaissent pas en ligne UDI sont uniquement des TTP et/ou CAP ?

query = """
SELECT cdreseauamont, nomreseauamont
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
    FROM edc_prelevements
    WHERE cdreseauamont IS NULL)
AND nomreseauamont NOT LIKE 'TTP%' AND nomreseauamont NOT LIKE 'CAP%'
GROUP BY 1, 2
ORDER BY 1;
"""

test_ttp = con.execute(query).fetch_df()
test_ttp

,cdreseauamont,nomreseauamont
0,001000301,TALISSIEU AMEYZIEU
1,001004228,SIE ST AMOUR-COLIGNY - ST AMOUR
2,002000118,BRECY
3,002000124,ANIZY LE GRAND
4,002000150,COINCY
...,...,...
19606,976000054,STATION PASSAMAINTY
19607,976000056,UP PAMANDZI
19608,976003465,STATION VAHIBE
19609,976003502,STATION CHIRONGUI


Les prélèvements qui ne proviennent pas d'UDI direct ne sont pas forcément des prélèvements réalisés uniquement sur des TTP ou CAP. Analyse plus poussée à réaliser et potentiellement questions à remonter à des experts...

Acronymes à élucider : ABA, UV (Station), TRT (Traitement), CHL (chlore), Livraison, Traitement, Réservoir, Neutralisation, MEL (Melange), Production, STP...

Les réseaux en amont sont à priori principalement des stations de traitement de l'eau (traitement final harmonisé avant distribution). 

In [36]:
# Est-ce que les cdreseauamont des prélèvements hors UDI "source" sont retrouvables quelque part dans la table ?

query = """
WITH cte AS (
SELECT cdreseau, cdreseauamont, referenceprel, nomreseauamont
FROM edc_prelevements
WHERE referenceprel NOT IN (
    SELECT referenceprel
    FROM edc_prelevements
    WHERE cdreseauamont IS NULL)
ORDER BY referenceprel
)

SELECT COUNT(*) AS nb_udi
FROM cte
JOIN edc_prelevements p ON cte.cdreseauamont = p.cdreseau
"""

test_2 = con.execute(query).fetch_df()
test_2

,nb_udi
0,6651


Les *350 280* prélèvements uniques hors UDI "source" ont tous un 'cdreseauamont'. Or, la majorité de ces 'cdreseauamont' ne sont pas retrouvables dans 'cdreseau', n'ont pas de ligne attitrée.

L'enquête nous mène à nous demander ce que ces prélèvements ont de spécial ? 
Est-ce parce que ces derniers ont lieu en amont (TTP / CAP) ? Est-ce pour une autre raison ?  

Quelle est la différence entre un prélèvement en direct sur UDI et les autres types de prélèvement ?

==> Faire l'analyse sur les différentes années 2020 -> 2024

### 3 - Analyse de la table edc_resultats